## Paso 5 - Entrenamiento con Aumento de Datos

En este notebook se desarrolla el punto 5 del caso práctico:

**"Modelo basado en aumento de datos sobre cualquiera de los cuatro modelos implementados anteriormente"**

Este notebook implementa el paso 5 de la práctica final, consistente en aplicar **aumento de datos (data augmentation)** sobre el modelo con mejor rendimiento anterior (en este caso, `transfer_model`), con el objetivo de mejorar su capacidad de generalización.

### Aumento de datos con `ImageDataGenerator`

Se aplican múltiples transformaciones en tiempo real durante el entrenamiento:

- `rotation_range=30` → rotaciones aleatorias
- `width_shift_range=0.1`, `height_shift_range=0.1` → desplazamientos horizontales/verticales
- `shear_range=0.2` → deformaciones angulares
- `zoom_range=0.2` → zoom aleatorio
- `horizontal_flip=True` → inversión horizontal
- `fill_mode='nearest'` → relleno de píxeles

### Reentrenamiento

- El modelo se recompila antes de entrenarlo con datos aumentados.
- Se utiliza `EarlyStopping` para evitar el sobreentrenamiento.







In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

from utils.dataloader import load_data_npy, DataGenerator
from utils.model_utils import save_model_and_history, save_test_results

In [ ]:
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy(resize_to(224, 224))

In [ ]:
# --- Aumento de datos con ImageDataGenerator ---
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valgen = ImageDataGenerator()
testgen = ImageDataGenerator()

train_aug = datagen.flow(images_train, categories_train, batch_size=32)
val_aug = valgen.flow(images_val, categories_val, batch_size=32)
test_aug = testgen.flow(images_test, categories_test, batch_size=32, shuffle=False)

# --- Cargar mejor modelo anterior ---
model, _ = load_model_and_history("transfer_model")

# --- Compilar de nuevo ---
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# --- Entrenar con datos aumentados ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_aug,
    validation_data=val_aug,
    epochs=30,
    callbacks=[early_stop]
)

# --- Evaluar en test ---
test_loss, test_acc = model.evaluate(test_aug)
print(f"\nTest accuracy con aumento de datos: {test_acc:.4f}   |  Test Loss: {test_loss:.4f}")

# --- Guardar nuevo modelo ---
save_model_and_history(model, history, model_path='transfer_model_augmented')
save_test_results('model_extended_cnn', test_loss, test_acc)